# Instructions

1. Make a copy of this notebook (EasyOCR.ipynb)
  * Upload the EasyOCR.ipynb to Google Colab
  * Click on "Copy to Drive"

2. Adjust input and output folders (IMAGE_DIR, OUTPUT_DIR)
  * Under the block "Manage Input and Output", change the following directories:
    *   IMAGE_DIR should be the path to a folder of your input images
    *   OUTPUT_DIR should be the desired folder location for the output txt files

3. Run the OCR Script
  * Click on "Runtime"
  * Click "Run all" to run all the cells



# Environment Setup

In [ ]:
!pip install easyocr

# Manage Input and Output

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
IMAGE_DIR = '/content/drive/My Drive/OCR/Input_OCR_Textures/'
OUTPUT_DIR = '/content/drive/My Drive/OCR/Output_OCR_Files/'

# Optical Character Recognition on Engraved Headstones

In [ ]:
import os
import cv2
import easyocr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import requests
from difflib import get_close_matches
from google.colab.patches import cv2_imshow

def preprocess_image(image):

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_equalized = cv2.equalizeHist(gray)
    gamma_corrected = cv2.pow(gray_equalized / 255.0, 1) #gamme
    gamma_corrected = (gamma_corrected * 255).astype(np.uint8)
    toned_down_img = cv2.convertScaleAbs(gamma_corrected, alpha=1.8, beta=0) #alpha, beta

    thresh = cv2.adaptiveThreshold(toned_down_img, 245, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, \
                                        cv2.THRESH_BINARY, 11, 2)

    return thresh

def download_english_words():
    url = "https://raw.githubusercontent.com/dwyl/english-words/master/words_alpha.txt"
    response = requests.get(url)
    english_words = set(response.text.split())

    return english_words

def extract_text(image, image_path):

    preprocessed_img = preprocess_image(image)

    reader = easyocr.Reader(['en'])
    result = reader.readtext(preprocessed_img)

    english_words = download_english_words()

    # Specify structure shape and kernel size
    rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 10))

    # Applying dilation on threshhold image
    dilation = cv2.dilate(preprocessed_img, rect_kernel, iterations = 1)

    # Finding contours
    contours, _ = cv2.findContours(dilation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    # Create a copy of the image
    im2 = image.copy()

    # Display bounding boxes around detected text
    fig, ax = plt.subplots(1)
    ax.imshow(cv2.cvtColor(preprocessed_img, cv2.COLOR_BGR2RGB))

    for contour in contours:
      # Bounding box detection (using contour detection)
      x, y, w, h = cv2.boundingRect(contour)
      rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')

      # Cropping the text box
      cropped = im2[y:y + h, x:x + w]

      # Run EasyOCR on cropped text
      result = reader.readtext(cropped)

      extracted_text = []
      for detection in result:
         # Bounding box coordinates (using text detection)
          points = detection[0]
          x, y, x1, y1 = points[0][0], points[0][1], points[2][0], points[2][1]
          w, h = x1 - x, y1 - y

          rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
          ax.add_patch(rect)

          # Find the closest English word
          detected_text = detection[1].lower()
          detected_text = detected_text.replace("[", "1").replace("(", "1")
          closest_word = get_close_matches(detected_text, english_words, n=1)[0] if detected_text in english_words else None

          extracted_text.append(closest_word.upper() if closest_word else detected_text.upper())
          # print(detection[1].upper())

    ax.set_title(os.path.basename(image_path))
    plt.show()
    return extracted_text if extracted_text else []

def process_images_in_folder(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(input_folder, filename)
            image = cv2.imread(image_path)
            extracted_text = extract_text(image, filename)

            output_file_path = os.path.join(output_folder, os.path.splitext(filename)[0] + ".txt")
            with open(output_file_path, "w") as txt_file:
                for line in extracted_text:
                    txt_file.write(line + "\n")

            print(f"Output produced for image: {filename}")

if __name__ == '__main__':
    process_images_in_folder(IMAGE_DIR, OUTPUT_DIR)
